In [ ]:
import sys
print("Hello from Colab kernel!", sys.version)


In [ ]:
import platform
print("Python:", platform.python_version())

!nvidia-smi


In [3]:
!pip install -q pandas numpy scikit-learn gradio requests openpyxl


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import gradio as gr
import requests


In [1]:
!git clone -b dev https://github.com/Abhishekchoure01/crop-risk-predictor.git
%cd crop-risk-predictor
!git pull origin dev
!pip install -q -r requirements.txt
!ls -la src/

fatal: destination path 'crop-risk-predictor' already exists and is not an empty directory.
/content/crop-risk-predictor
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 30 (delta 11), reused 28 (delta 9), pack-reused 0 (from 0)
Unpacking objects: 100% (30/30), 8.60 KiB | 1.23 MiB/s, done.
From https://github.com/Abhishekchoure01/crop-risk-predictor
 * branch            dev        -> FETCH_HEAD
   6bd0ea5..1cd320e  dev        -> origin/dev
Updating 6bd0ea5..1cd320e
Fast-forward
 notebooks/crop_risk_colab.ipynb | 172 ++++++++++++++++++++++++++++++++++++++++
 requirements.txt                |   6 ++
 src/__init__.py                 |  11 +++
 src/app.py                      | 132 ++++++++++++++++++++++++++++++
 src/data_gen.py                 |  47 +++++++++++
 src/model.py                    | 114 ++++++++++++++++++++++++++
 src/risk_logic.py               |  75 ++++++++++++++++++
 7 files

In [4]:
import sys
import os

# Add the cloned repo to Python path
sys.path.insert(0, '/content/crop-risk-predictor')

from src.data_gen import generate_synthetic_data
from src.model import CropRiskModel
from src.risk_logic import explain_risk
from src.app import create_gradio_app

print("✓ All imports successful!")

✓ All imports successful!


In [5]:
# 1) Synthetic data
rng = np.random.default_rng(42)

def generate_synthetic_data(n=50):
    rows = []
    for _ in range(n):
        rainfall_pct = 100 + rng.normal(0, 20)
        heatwave_days = 4 + rng.normal(0, 3)
        dry_days = 10 + rng.normal(0, 5)
        humidity = 70 + rng.normal(0, 8)
        loss = (100 - rainfall_pct) * 0.4 + heatwave_days * 5 + dry_days * 1.5 + max(0, humidity - 70) * 0.3
        rows.append([rainfall_pct, heatwave_days, dry_days, humidity, loss])
    df = pd.DataFrame(rows, columns=["rainfall_pct","heatwave_days","dry_days","humidity","loss_pct"])
    return df

df = generate_synthetic_data()
df.head()


,rainfall_pct,heatwave_days,dry_days,humidity,loss_pct
0,106.094342,0.880048,13.752256,77.524518,24.848241
1,60.979296,0.093461,10.639202,67.470059,32.034392
2,99.663977,1.440868,14.396990,76.222335,30.800936
3,101.320614,7.381724,12.337547,63.125660,54.886693
4,107.375016,1.123352,14.392252,69.600593,24.255132


In [6]:
# 2) Train model
features = ["rainfall_pct","heatwave_days","dry_days","humidity"]
X = df[features]
y = df["loss_pct"]

model = LinearRegression().fit(X, y)
r2 = r2_score(y, model.predict(X))
r2


0.9985378346965189

In [ ]:
# ============================================================================
# DISTRICTS, CROPS, and WEATHER DATA (UNCHANGED)
# ============================================================================
DISTRICTS = ["Pune","Nagpur","Mumbai"]
CROPS = ["Rice","Wheat","Cotton"]

DISTRICT_WEATHER = {
    "Pune":  {"rainfall_pct": 100, "heatwave_days": 4, "dry_days": 10, "humidity": 70},
    "Nagpur":{"rainfall_pct": 95,  "heatwave_days": 6, "dry_days": 12, "humidity": 68},
    "Mumbai":{"rainfall_pct": 120, "heatwave_days": 2, "dry_days": 5,  "humidity": 80},
}

# ============================================================================
# PREDICT_RISK FUNCTION (UNCHANGED)
# ============================================================================
def predict_risk(district, crop):
    w = DISTRICT_WEATHER[district]
    X_pred = np.array([[w["rainfall_pct"], w["heatwave_days"], w["dry_days"], w["humidity"]]])
    loss = float(model.predict(X_pred)[0])
    
    contrib = {
        "rainfall_pct": max(0, (100 - w["rainfall_pct"]) * 0.4),
        "heatwave_days": w["heatwave_days"] * 5,
        "dry_days": w["dry_days"] * 1.5,
        "humidity": max(0, (w["humidity"] - 70) * 0.3),
    }
    top2 = sorted(contrib.items(), key=lambda x: x[1], reverse=True)[:2]
    severity = "🔴 HIGH" if loss > 30 else "🟡 MEDIUM" if loss > 15 else "🟢 LOW"
    
    lines = [
        f"{district} - {crop} Risk Assessment",
        f"🎯 Predicted Loss: {loss:.0f}% {severity}",
        "🔴 Top Risks:",
    ]
    for name, val in top2:
        lines.append(f"• {name}: {val:.0f}%")
    lines.append("✅ Actions:")
    lines.append("1. Switch to drought-resistant variety")
    lines.append("2. Prepare for early harvest")
    lines.append(f"📊 Model Confidence: R² = {r2:.2f}")
    return "\n".join(lines)

# ============================================================================
# TRANSLATIONS
# ============================================================================
TRANSLATIONS = {
    "English": {
        "title": "🌾 Crop Risk Prediction",
        "subtitle": "AI-Powered Crop Risk Assessment for Indian Farmers",
        "hero_desc": "Protect your harvest with intelligent weather-based risk analysis",
        "district": "Select District",
        "crop": "Select Crop",
        "language": "Language",
        "predict_btn": "🔮 Predict Risk",
        "results": "Risk Assessment Results",
        "select_inputs": "👈 Select district, crop, and language to begin",
    },
    "मराठी": {
        "title": "🌾 पिकाचा जोखीम अंदाज",
        "subtitle": "भारतीय शेतकऱ्यांसाठी AI-संचालित पिकाचा जोखीम मुल्यांकन",
        "hero_desc": "हवामान-आधारित जोखीम विश्लेषणासह आपली कसद संरक्षित करा",
        "district": "जिल्हा निवडा",
        "crop": "पिक निवडा",
        "language": "भाषा",
        "predict_btn": "🔮 जोखीम अंदाज",
        "results": "जोखीम मूल्यांकन परिणाम",
        "select_inputs": "👈 सुरू करण्यासाठी जिल्हा, पिक आणि भाषा निवडा",
    },
    "हिन्दी": {
        "title": "🌾 फसल जोखिम भविष्यवाणी",
        "subtitle": "भारतीय किसानों के लिए AI-संचालित फसल जोखिम आकलन",
        "hero_desc": "मौसम-आधारित जोखिम विश्लेषण के साथ अपनी फसल की रक्षा करें",
        "district": "जिला चुनें",
        "crop": "फसल चुनें",
        "language": "भाषा",
        "predict_btn": "🔮 जोखिम पूर्वानुमान",
        "results": "जोखिम मूल्यांकन परिणाम",
        "select_inputs": "👈 शुरू करने के लिए जिला, फसल और भाषा चुनें",
    },
}

# ============================================================================
# ENHANCED GRADIO APP WITH GLASS MORPHISM & MULTILINGUAL SUPPORT
# ============================================================================
def create_enhanced_app():
    # Create a custom theme
    custom_theme = gr.themes.Soft(primary_hue="green", secondary_hue="blue")
    
    with gr.Blocks(theme=custom_theme, css="""
    body {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    }
    .glass-card, .gradio-group {
        backdrop-filter: blur(10px);
        background: rgba(255, 255, 255, 0.15) !important;
        border: 1px solid rgba(255, 255, 255, 0.3) !important;
        border-radius: 15px;
        padding: 20px;
        box-shadow: 0 8px 32px rgba(31, 38, 135, 0.37);
    }
    .hero-section {
        text-align: center;
        padding: 40px 20px;
        background: linear-gradient(135deg, rgba(255,255,255,0.15) 0%, rgba(255,255,255,0.05) 100%);
        border-radius: 20px;
        border: 1px solid rgba(255,255,255,0.3);
        margin-bottom: 30px;
        box-shadow: 0 8px 32px rgba(31, 38, 135, 0.37);
    }
    .hero-title {
        font-size: 2.5em;
        font-weight: bold;
        margin-bottom: 10px;
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        background-clip: text;
        color: transparent;
    }
    .hero-subtitle {
        font-size: 1.1em;
        color: rgba(255,255,255,0.9);
        margin-bottom: 15px;
        font-weight: 500;
    }
    .result-card {
        background: linear-gradient(135deg, rgba(102, 126, 234, 0.15) 0%, rgba(118, 75, 162, 0.15) 100%);
        border: 2px solid rgba(102, 126, 234, 0.4) !important;
        border-radius: 15px;
        padding: 25px;
        font-family: 'Courier New', monospace;
        backdrop-filter: blur(10px);
        box-shadow: 0 8px 32px rgba(31, 38, 135, 0.37);
    }
    .gradio-button {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
        font-size: 1.1em;
        padding: 12px 30px;
        font-weight: bold;
        border-radius: 10px;
        border: none;
        color: white;
    }
    .gradio-button:hover {
        opacity: 0.9;
        transform: translateY(-2px);
        box-shadow: 0 12px 24px rgba(102, 126, 234, 0.4);
    }
    """) as demo:
        
        # ====== HERO SECTION ======
        with gr.Group(elem_classes="hero-section"):
            gr.HTML("<div class='hero-title'>🌾 Crop Risk Prediction</div>")
            gr.HTML("<div class='hero-subtitle'>AI-Powered Assessment for Indian Farmers</div>")
            gr.HTML("<div style='color: rgba(255,255,255,0.85); font-size: 1em;'>Protect your harvest with intelligent weather-based risk analysis</div>")
        
        # ====== CONTROLS SECTION ======
        with gr.Group(elem_classes="glass-card"):
            gr.Markdown("### ⚙️ Configuration", elem_classes="")
            lang_dd = gr.Dropdown(
                choices=list(TRANSLATIONS.keys()),
                value="English",
                label="Language / भाषा / ভাষা",
                scale=1
            )
        
        with gr.Group(elem_classes="glass-card"):
            gr.Markdown("### 📍 Select Your Region & Crop")
            with gr.Row():
                dist_dd = gr.Dropdown(
                    DISTRICTS,
                    label="District",
                    scale=1,
                    interactive=True
                )
                crop_dd = gr.Dropdown(
                    CROPS,
                    label="Crop",
                    scale=1,
                    interactive=True
                )
        
        # ====== PREDICTION BUTTON ======
        predict_btn = gr.Button(
            "🔮 Predict Risk",
            variant="primary",
            scale=2
        )
        
        # ====== RESULTS SECTION ======
        with gr.Group(elem_classes="result-card"):
            gr.Markdown("### 📊 Risk Assessment Results")
            result_output = gr.Markdown(
                "👈 Select district, crop, and language to begin",
                label="Results"
            )
        
        # ====== FOOTER ======
        gr.Markdown(
            "---\n"
            "🔬 **Model Performance**: R² = {:.2f} | "
            "🌐 **Languages**: English • मराठी • हिन्दी | "
            "📱 **Empowering Indian Farmers**".format(r2)
        )
        
        # ====== EVENT HANDLERS ======
        def update_on_predict(district, crop, lang):
            if not district or not crop:
                return "👈 Please select both district and crop to proceed"
            return predict_risk(district, crop)
        
        predict_btn.click(
            fn=update_on_predict,
            inputs=[dist_dd, crop_dd, lang_dd],
            outputs=result_output
        )
        
        return demo

# ====== LAUNCH APP ======
app = create_enhanced_app()
app.launch(share=True)

TypeError: Base.set() got an unexpected keyword argument 'body_background_fill_secondary'